In [13]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse
import pandas as pd
import pyspark
from pyspark.sql.types import *
from pyspark.sql.functions import pandas_udf
import pickle as pkl
import pyspark
from pyspark.sql.functions import pandas_udf




In [2]:
df = pd.read_parquet('src\df_final.parquet')
df.head()

,userId,score,timestamp,id,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma,duration_int,duration_type
0,1,1.0,1425941529,as680,s680,tv show,the english civil war,None,robert whelan,None,None,2019,13+,1 season,"documentary, special interest",a nation divided. it was a time of great bitte...,a,1,season
1,583,4.5,1112982383,as680,s680,tv show,the english civil war,None,robert whelan,None,None,2019,13+,1 season,"documentary, special interest",a nation divided. it was a time of great bitte...,a,1,season
2,765,5.0,889677014,as680,s680,tv show,the english civil war,None,robert whelan,None,None,2019,13+,1 season,"documentary, special interest",a nation divided. it was a time of great bitte...,a,1,season
3,2116,3.0,836264839,as680,s680,tv show,the english civil war,None,robert whelan,None,None,2019,13+,1 season,"documentary, special interest",a nation divided. it was a time of great bitte...,a,1,season
4,2143,3.0,1435067487,as680,s680,tv show,the english civil war,None,robert whelan,None,None,2019,13+,1 season,"documentary, special interest",a nation divided. it was a time of great bitte...,a,1,season


In [3]:
reader = Reader(rating_scale=(1, 5))


In [4]:
# Crear el dataset de Surprise
data = Dataset.load_from_df(df[['userId', 'id', 'score']], reader)

In [5]:
# Dividir los datos en train y test
trainset, testset = train_test_split(data, test_size=0.2)

In [6]:
# Crear el modelo y entrenarlo con los datos de train
model = SVD()
model.fit(trainset)

In [8]:
# Predecir las puntuaciones para los datos de test
predictions = model.test(testset)

In [9]:
# Calcular el error RMSE
accuracy = rmse(predictions)
print('RMSE:', accuracy)

RMSE: 1.0009
RMSE: 1.0008635003183566


In [10]:
spark = pyspark.sql.SparkSession.builder.appName('predicions_movie').getOrCreate()


In [11]:
schema = StructType([
    StructField('modelo', BinaryType(), True)
])


In [14]:
#Guardar el modelo entrenado en un archivo utilizando pickle
with open('mi_modelo_entrenado.pkl', 'wb') as archivo:
    pkl.dump(model, archivo)

In [18]:
userId = '4'
itemId = 'ns8'
prediction = model.predict(userId, itemId)

if prediction.est >=4:
    mensaje = 'esta muy buena, hay que verla', prediction.est
elif prediction.est >=3 and prediction.est < 4:
    mensaje = 'es buena pelicula', prediction.est
else:
    mensaje = 'es una pelicula muy "hueso"'

print(mensaje)


('es buena pelicula', 3.5646862928357925)
